## This notebook:
### reads three .S2P file from three measurements of 5W30  synthetic oil;
### creates a DataFrame pandas object from the info of .S2P file
### selects the desired features and saves a .CSV file.


#### Imports

In [37]:
import os
import re
import skrf as rf
import pandas as pd

#### Paths

In [38]:
script_directory = os.getcwd()#obtem diretório atual
s2ps_path = os.path.join(script_directory,'..','..','oil_data','oil_data') #cria string caminho para todos os .s2p

#### Regex

In [39]:
regex = r'SAW_06_02_2024_OIL_5W30_(\d+)\.S2P'

In [40]:
def read_s2p_data(regex, path):
    dfs = []#lista para ocncatenar todos os dicts de cada medição 
    for filename in os.listdir(path):
        match = re.match(regex, filename)
        if match:
            #obtem informação do nome do arquivo
            oil_name = "5W30"
            measurement_number = int(match.group(1))
            
            #carrega arquivo
            path_to_oil = os.path.join(path, filename)
            s2p_file = rf.Network(path_to_oil)
            
            #extrai frequência, parâmetro S21 e fase do objeto Network
            frequency = s2p_file.f
            s_parameters = s2p_file.s_db[:,1,0]
            phase = s2p_file.s_deg[:,1,0]
            
            #Dataframe com informações selecionadas
            individual_oil_dict = {
                'OIL NAME': oil_name,
                'OIL TYPE': 'SYNTHETIC',
                'MEASUREMENT NUMBER': measurement_number,
                'FREQUENCY (MHZ)':frequency/1000000,
                'S21 (db)' : s_parameters,
                'PHASE (DEG)': phase,
            }
            df = pd.DataFrame.from_dict(individual_oil_dict)
            df = df.nlargest(3, 'S21 (db)')
            dfs.append(df)
            
    dataframe = pd.concat(dfs, ignore_index=True)
    return dataframe

#### Read Dataframes

In [41]:
df_synthetic = read_s2p_data(regex, s2ps_path)

In [42]:
df_synthetic.info

<bound method DataFrame.info of   OIL NAME   OIL TYPE  MEASUREMENT NUMBER  FREQUENCY (MHZ)  S21 (db)  \
0     5W30  SYNTHETIC                   1       118.872887 -57.27288   
1     5W30  SYNTHETIC                   1       118.888889 -57.45019   
2     5W30  SYNTHETIC                   1       118.890889 -57.45746   
3     5W30  SYNTHETIC                   2       118.888889 -56.54844   
4     5W30  SYNTHETIC                   2       118.892889 -56.77471   
5     5W30  SYNTHETIC                   2       118.890889 -56.84834   
6     5W30  SYNTHETIC                   3       118.874887 -56.77548   
7     5W30  SYNTHETIC                   3       118.886889 -56.79126   
8     5W30  SYNTHETIC                   3       118.858886 -56.88812   

   PHASE (DEG)  
0    -162.4664  
1    -173.8333  
2    -173.8878  
3    -175.7385  
4    -176.5249  
5    -174.9888  
6    -163.4336  
7    -172.6035  
8    -151.9654  >

In [43]:
#assegura somente óleo 5W30 lido
for name in df_synthetic["OIL NAME"]:
    if name != "5W30":
        print("Diferente de 5W30")

In [44]:
path_save_csv = os.path.join(script_directory,'..','..', 'oil_data','oil_data','CSVs')
if not os.path.exists(path_save_csv):
   os.makedirs(path_save_csv)
df_synthetic.to_csv(path_save_csv + '/5W30_measurements.csv')

--------------------------------------------------------------------------

In [45]:
df_synthetic.info

<bound method DataFrame.info of   OIL NAME   OIL TYPE  MEASUREMENT NUMBER  FREQUENCY (MHZ)  S21 (db)  \
0     5W30  SYNTHETIC                   1       118.872887 -57.27288   
1     5W30  SYNTHETIC                   1       118.888889 -57.45019   
2     5W30  SYNTHETIC                   1       118.890889 -57.45746   
3     5W30  SYNTHETIC                   2       118.888889 -56.54844   
4     5W30  SYNTHETIC                   2       118.892889 -56.77471   
5     5W30  SYNTHETIC                   2       118.890889 -56.84834   
6     5W30  SYNTHETIC                   3       118.874887 -56.77548   
7     5W30  SYNTHETIC                   3       118.886889 -56.79126   
8     5W30  SYNTHETIC                   3       118.858886 -56.88812   

   PHASE (DEG)  
0    -162.4664  
1    -173.8333  
2    -173.8878  
3    -175.7385  
4    -176.5249  
5    -174.9888  
6    -163.4336  
7    -172.6035  
8    -151.9654  >